In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt


In [3]:

# 4. Load and Prepare Data
data = pd.read_csv('vehicle_anomaly_dataset.csv')
X = data[['gyro_x', 'gyro_y', 'gyro_z', 'pedometer', 'radar_distance', 'radar_velocity', 'potentiometer_angle']]
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
# 5. Train Isolation Forest
iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
X_train_normal = X_train[y_train == 0]
iso_forest.fit(scaler.transform(X_train_normal))
y_pred = iso_forest.predict(X_test_scaled)
y_pred = np.where(y_pred == 1, 0, 1)

# 6. Evaluate Isolation Forest
print("Isolation Forest Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("ROC AUC Score:")
print(roc_auc_score(y_test, iso_forest.decision_function(X_test_scaled)))

Isolation Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1980
           1       0.49      1.00      0.66        20

    accuracy                           0.99      2000
   macro avg       0.74      0.99      0.83      2000
weighted avg       0.99      0.99      0.99      2000

Confusion Matrix:
[[1959   21]
 [   0   20]]
ROC AUC Score:
0.0


In [6]:

# 7. Plot Reconstruction Error for Autoencoder
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

autoencoder = Sequential([
    Dense(4, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(X_train_scaled.shape[1], activation='sigmoid')
])
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
autoencoder.fit(X_train_normal, X_train_normal, epochs=50, batch_size=32, validation_split=0.1, shuffle=True)

X_test_pred = autoencoder.predict(X_test_scaled)
reconstruction_error = np.mean(np.abs(X_test_scaled - X_test_pred), axis=1)
threshold = np.percentile(reconstruction_error, 95)
y_pred_auto = (reconstruction_error > threshold).astype(int)

print("\nAutoencoder Classification Report:")
print(classification_report(y_test, y_pred_auto))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_auto))
print("ROC AUC Score:")
print(roc_auc_score(y_test, reconstruction_error))

Epoch 1/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 485.7455 - val_loss: 485.4828
Epoch 2/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step - loss: 484.0087 - val_loss: 485.1622
Epoch 3/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - loss: 483.7887 - val_loss: 485.1238
Epoch 4/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 863us/step - loss: 484.4341 - val_loss: 484.8170
Epoch 5/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - loss: 483.8106 - val_loss: 484.7157
Epoch 6/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - loss: 484.1564 - val_loss: 484.6753
Epoch 7/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - loss: 482.2752 - val_loss: 484.6459
Epoch 8/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step - loss: 484.6439 - val_loss: 484.6296
Epoch 9/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step - loss: 487.4138 - val_loss: 484.6175
Epoch 10/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - loss: 484.9066 - val_loss: 484.6083
Epoch 11/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - loss: 480.0843 - val_los